<a href="https://colab.research.google.com/github/nadfai/m22ai582/blob/main/finetune_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune the model

In [1]:
from keras.applications import VGG16
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Setting the input size of the images
img_width, img_height = 224, 224

In [14]:
# Setting the directories of the training and validation data
train_data_dir = "/content/drive/MyDrive/Colab Notebooks/train-20230429T125055Z-001"
val_data_dir = "/content/drive/MyDrive/Colab Notebooks/val-20230429T125056Z-001"

In [15]:
# Creating an instance of the VGG16 model with pre-trained weights
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [16]:
# Freezing the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [17]:
# Adding new layers to the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

In [18]:
# Defining the new model with the pre-trained model as its base and the new layers as its top
model = Model(inputs=base_model.input, outputs=predictions)

In [19]:
# Compiling the model with a binary crossentropy loss and an Adam optimizer
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-5), metrics=['accuracy'])

In [20]:
# Setting up data augmentation for the training data and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [21]:
# Setting the batch size
batch_size = 16

# Setting the number of training and validation samples
nb_train_samples = 1600
nb_val_samples = 400

# Setting the number of epochs
epochs = 10

In [22]:
# Training the model with the data generators
history = model.fit(
    train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary'),
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary'),
    validation_steps=nb_val_samples // batch_size)

Found 1000 images belonging to 1 classes.
Found 178 images belonging to 1 classes.


/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:202: UserWarning: Using ".tiff" files with multiple bands will cause distortion. Please verify your output.
  warnings.warn(


Epoch 1/10
 63/100 [=================>............] - ETA: 6:30 - loss: 0.0177 - accuracy: 1.0000

100/100 [==============================] - 813s 8s/step - loss: 0.0177 - accuracy: 1.0000 - val_loss: 4.2094e-04 - val_accuracy: 1.0000


In [23]:
# Evaluate the model on the test data
test_datagen = ImageDataGenerator(rescale=1./255)
test_data_dir = "/content/drive/MyDrive/Colab Notebooks/test"

In [24]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 0 images belonging to 0 classes.


In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))

In [ ]:
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)